# Education, Wage, and Quarter of Birth

In this review session, we are going to practice using the different regression approaches we talked about in class to study the relationship between wage, education, and quarter of birth. We are going to use OLS, instrumental variables, fixed effects to estimate the return to education, and logistic regression to estimate how quarter of birth affects the probability of someone getting more than 12 years of education. You can check out [Angrist and Krueger (1991)](https://www.jstor.org/stable/2937954) if you are interested in learning more about this topic. 

Before we start, let's load the libraries we are going to use

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.sandbox.regression.gmm import IV2SLS
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

We will then load the dataset. This dataset contains information on weekly wages, education, quarter and year of birth for men born 1920-29 in 1970. The raw dataset can be found [here](https://economics.mit.edu/faculty/angrist/data1/data/angkru1991).

In [2]:
df = pd.read_stata('data/QOB7080_clean.dta')
df.columns

Index(['AGE', 'AGEQ', 'v3', 'EDUC', 'ENOCENT', 'ESOCENT', 'v7', 'LWKLYWGE',
       'MARRIED', 'MIDATL', 'MT', 'NEWENG', 'v14', 'v15', 'CENSUS', 'v17',
       'QOB', 'RACE', 'SMSA', 'SOATL', 'v22', 'v23', 'WNOCENT', 'WSOCENT',
       'v26', 'YOB', 'COHORT', 'AGEQSQ', 'YR20', 'YR21', 'YR22', 'YR23',
       'YR24', 'YR25', 'YR26', 'YR27', 'YR28', 'YR29', 'QTR1', 'QTR2', 'QTR3',
       'QTR4', 'QTR120', 'QTR121', 'QTR122', 'QTR123', 'QTR124', 'QTR125',
       'QTR126', 'QTR127', 'QTR128', 'QTR129', 'QTR220', 'QTR221', 'QTR222',
       'QTR223', 'QTR224', 'QTR225', 'QTR226', 'QTR227', 'QTR228', 'QTR229',
       'QTR320', 'QTR321', 'QTR322', 'QTR323', 'QTR324', 'QTR325', 'QTR326',
       'QTR327', 'QTR328', 'QTR329', 'QTR420', 'QTR421', 'QTR422', 'QTR423',
       'QTR424', 'QTR425', 'QTR426', 'QTR427', 'QTR428', 'QTR429'],
      dtype='object')

## OLS

Now, we want to figure out what the return on education is. A naive approach would simply be regressing log weekly ages on years of education:

$$\ln W_i = X_i \beta + \sum_c Y_{ic}\xi_c + \rho E_i + \mu_i$$

We will first set up the list of regressors.

In [3]:
# Creating the list of regressors
YOB = ['YR2' + str(i) for i in range(9)]
QTR = ['QTR' + str(qob) + '2' + str(yob) for qob in range(1, 4) for yob in range(10)]
# Controlling for race, marital status, standrad metropolitan statistical areas, and regions of residence
controls = ['RACE', 'MARRIED', 'SMSA',
            'NEWENG', 'MIDATL', 'ENOCENT',
            'WNOCENT', 'SOATL', 'ESOCENT',
            'WSOCENT', 'MT']

We now run four specifications of regression.

In [4]:
specs = {}
specs['spec1'] = sm.add_constant(df[['EDUC'] + YOB])
specs['spec2'] = sm.add_constant(df[['EDUC'] + YOB + ['AGEQ', 'AGEQSQ']])
specs['spec3'] = sm.add_constant(df[['EDUC'] + YOB + controls])
specs['spec4'] = sm.add_constant(df[['EDUC'] + YOB + controls + ['AGEQ', 'AGEQSQ']])

In [5]:
OLS_specs = {key:sm.OLS(df['LWKLYWGE'], specs[key]).fit() for key in specs.keys()}

We can look at the result for spec 1:

In [6]:
OLS_specs['spec1'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               LWKLYWGE   R-squared:                       0.171
Model:                            OLS   Adj. R-squared:                  0.171
Method:                 Least Squares   F-statistic:                     5101.
Date:                Sat, 12 Feb 2022   Prob (F-statistic):               0.00
Time:                        08:51:05   Log-Likelihood:            -2.2157e+05
No. Observations:              247199   AIC:                         4.432e+05
Df Residuals:                  247188   BIC:                         4.433e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.2100      0.006    749.545      0.000       4.199       4.221
EDUC           0.0802      0.000    225.670      0.000       0.079       0.081
YR20           0.0235      0.005      4.359      0.000       0.013       0.034
YR21           0.0290      0.005      5.453      0.000       0.019       0.039
YR22           0.0232      0.005      4.340      0.000       0.013       0.034
YR23           0.0256      0.005      4.777      0.000       0.015       0.036
YR24           0.0264      0.005      5.000      0.000       0.016       0.037
YR25           0.0308      0.005      5.793      0.000       0.020       0.041
YR26           0.0291      0.005      5.445      0.000       0.019       0.040
YR27           0.0271      0.005      5.133      0.000       0.017       0.037
YR28           0.0243      0.005      4.547      0.000       0.014       0.035
==============================================================================
Omnibus:                   127757.844   Durbin-Watson:                   1.845
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1888557.827
Skew:                          -2.144   Prob(JB):                         0.00
Kurtosis:                      15.844   Cond. No.                         126.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

An useful thing can be to convert the result tables into pandas dataframe:

In [7]:
OLS_spec1_as_html = OLS_specs['spec1'].summary().tables[1].as_html()
# Check out why it returns a list of dfs
pd.read_html(OLS_spec1_as_html, header=0, index_col=0)[0]

,coef,std err,t,P>|t|,[0.025,0.975]
const,4.2100,0.006,749.545,0.0,4.199,4.221
EDUC,0.0802,0.000,225.670,0.0,0.079,0.081
YR20,0.0235,0.005,4.359,0.0,0.013,0.034
YR21,0.0290,0.005,5.453,0.0,0.019,0.039
YR22,0.0232,0.005,4.340,0.0,0.013,0.034
YR23,0.0256,0.005,4.777,0.0,0.015,0.036
YR24,0.0264,0.005,5.000,0.0,0.016,0.037
YR25,0.0308,0.005,5.793,0.0,0.020,0.041
YR26,0.0291,0.005,5.445,0.0,0.019,0.040
YR27,0.0271,0.005,5.133,0.0,0.017,0.037


When you have multiple specifications, it could be useful to write a function that does it for all specifications:

In [8]:
def res_tab_to_df(OLS_res, spec_name):
    """ This function converts output tables to pandas dataframes
    Args:
        OLS_res(sm.OLS object): output from OLS
        spec_name(str): a string indicating which specification the coefficients are from
    Returns:
        a pandas dataframe containing all coefficient estimates, t-statistics etc. 
    """
    OLS_res_as_html = OLS_res.summary().tables[1].as_html()
    OLS_res_df = pd.read_html(OLS_res_as_html, header=0, index_col=0)[0]
    OLS_res_df.loc[:, 'spec name'] = spec_name
    return OLS_res_df

In [9]:
OLS_res_df = pd.concat([res_tab_to_df(value, key) for key, value in OLS_specs.items()]).rename_axis('var').reset_index()
OLS_res_df[OLS_res_df['var']=='EDUC']

,var,coef,std err,t,P>|t|,[0.025,0.975],spec name
1,EDUC,0.0802,0.0,225.670,0.0,0.079,0.081,spec1
12,EDUC,0.0802,0.0,225.645,0.0,0.079,0.081,spec2
25,EDUC,0.0701,0.0,197.713,0.0,0.069,0.071,spec3
47,EDUC,0.0701,0.0,197.686,0.0,0.069,0.071,spec4


Also, note that this regression can be problematic if some variables are ommited, and $\mu$ is correlated with years of education. One of the possible reasons is that information on one's family's social class is not included in the regression.

## IV

To avoid this concern, we want to introduce an instrumental variable, which is three quarter-of-birth dummies interacted with year-of-birth dummies. The general idea is that quarter of birth is correlated with education, but not with the residual. We use three quarter-of-birth dummies interacted with year-of-birth dummies as our instruments, since we also include year of birth in our estimation, so the instruments will be capturing variations in education across quarters within the same birth cohort. See the paper for more discussion of why quarter of birth matters for education. 

We want to estimate the following relationship: 

$$E_i = X_i \pi + \sum_c Y_{ic}\delta_c + \sum_c\sum_j Y_{ic}Q_{ij}\theta_{jc} + \epsilon_I$$
$$\ln W_i = X_i \beta + \sum_c Y_{ic}\xi_c + \rho E_i + \mu_i$$


In [10]:
IV_specs = {}
IV_specs['spec1'] = sm.add_constant(df[QTR + YOB])
IV_specs['spec2'] = sm.add_constant(df[QTR + YOB + ['AGEQ', 'AGEQSQ']])
IV_specs['spec3'] = sm.add_constant(df[QTR + YOB + controls])
IV_specs['spec4'] = sm.add_constant(df[QTR + YOB + controls + ['AGEQ', 'AGEQSQ']])

In [11]:
IV_specs = {key:IV2SLS(df['LWKLYWGE'], specs[key], instrument=IV_specs[key]).fit() for key in specs.keys()}
IV_specs['spec1'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          IV2SLS Regression Results                           
==============================================================================
Dep. Variable:               LWKLYWGE   R-squared:                       0.171
Model:                         IV2SLS   Adj. R-squared:                  0.171
Method:                     Two Stage   F-statistic:                     10.43
                        Least Squares   Prob (F-statistic):           7.41e-18
Date:                Sat, 12 Feb 2022                                         
Time:                        08:51:08                                         
No. Observations:              247199                                         
Df Residuals:                  247188                                         
Df Model:                          10                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.2487      0.177     24.065      0.000       3.903       4.595
EDUC           0.0769      0.015      5.110      0.000       0.047       0.106
YR20           0.0218      0.010      2.286      0.022       0.003       0.040
YR21           0.0278      0.008      3.595      0.000       0.013       0.043
YR22           0.0220      0.008      2.833      0.005       0.007       0.037
YR23           0.0246      0.007      3.638      0.000       0.011       0.038
YR24           0.0257      0.006      4.147      0.000       0.014       0.038
YR25           0.0301      0.006      4.732      0.000       0.018       0.043
YR26           0.0286      0.006      4.968      0.000       0.017       0.040
YR27           0.0265      0.006      4.418      0.000       0.015       0.038
YR28           0.0239      0.006      4.291      0.000       0.013       0.035
==============================================================================
Omnibus:                   127902.693   Durbin-Watson:                   1.840
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1892549.789
Skew:                          -2.147   Prob(JB):                         0.00
Kurtosis:                      15.857   Cond. No.                         126.
==============================================================================
"""

We can apply the function we wrote before to look at the return to education for all these specifications. 

In [12]:
IV_res_df = pd.concat([res_tab_to_df(value, key) for key, value in IV_specs.items()]).rename_axis('var').reset_index()
IV_res_df[IV_res_df['var']=='EDUC']

,var,coef,std err,t,P>|t|,[0.025,0.975],spec name
1,EDUC,0.0769,0.015,5.110,0.000,0.047,0.106,spec1
12,EDUC,0.1310,0.033,3.929,0.000,0.066,0.196,spec2
25,EDUC,0.0669,0.015,4.430,0.000,0.037,0.096,spec3
47,EDUC,0.1007,0.033,3.014,0.003,0.035,0.166,spec4


## Binary Logistic Regression

Now, I will give an example of binary logistic regression. Our issue of interest is how the log odds for someone having an education of 12 years and more change based on their quarter of birth:

$$\text{logit}(educ_{12}) = X_i \pi + \sum_c\sum_j Y_{ic}Q_{ij}\theta_{jc} + \epsilon_I$$

In [13]:
df['EDUC_12'] = df['EDUC'] >= 12
logit_res = sm.Logit(df['EDUC_12'], df[['AGEQ', 'AGEQSQ', 'QTR1']]).fit(method='newton')
logit_res.summary()

Optimization terminated successfully.
         Current function value: 0.676300
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                EDUC_12   No. Observations:               247199
Model:                          Logit   Df Residuals:                   247196
Method:                           MLE   Df Model:                            2
Date:                Sat, 12 Feb 2022   Pseudo R-squ.:               0.0005406
Time:                        08:51:09   Log-Likelihood:            -1.6718e+05
converged:                       True   LL-Null:                   -1.6727e+05
Covariance Type:            nonrobust   LLR p-value:                 5.317e-40
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
AGEQ           0.0344      0.001     23.855      0.000       0.032       0.037
AGEQSQ        -0.0006   3.17e-05    -18.141      0.000      -0.001      -0.001
QTR1          -0.0471      0.009     -5.004      0.000      -0.066      -0.029
==============================================================================
"""